In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from time_series.dataset.utils import get_energy_demand
from time_series.dataset.time_series import TrainingDataSet
from time_series.utils import evaluate_model

train_df = get_energy_demand()
tds = TrainingDataSet(train_df)

In [16]:
from statsmodels.tsa.api import ExponentialSmoothing
from joblib import Parallel, delayed


def univariate_forecast(column):
    fit1 = ExponentialSmoothing(
        train_df[column].values[:split_point],
        seasonal_periods=4,
        trend="add",
        seasonal="add",
        use_boxcox=False,
        initialization_method="estimated",
    ).fit()
    return fit1.forecast(10)


split_point = int(len(train_df) * tds.train_split)
forecasts = Parallel(n_jobs=10)(
    delayed(univariate_forecast)(column) for column in train_df.columns
)

In [30]:
import numpy as np
from sklearn.metrics import mean_squared_error

actual = train_df.values[split_point:][:10, :]
round(mean_squared_error(actual, np.transpose(np.array(forecasts))), 2)

11.28